this is the final python script to export a single CSV file, without spatial information

In [ ]:
import os
import pyodbc
import csv
# [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]
# if you see [], you may need to install the 64-bit ACE drivers
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
# 64bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255

In [ ]:
access_database = r'Z:\dev\nepal-import\NepalInfo2016_for_python.accdb'
output_folder = 'nepal'
output_csv = 'devinfo_output.csv'

In [ ]:
connStr = (
    r"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};"
    r"DBQ={};".format(access_database)
    )

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT *
FROM ut_data_query 
ORDER BY indicator_nid ASC
"""

crsr = cnxn.execute(sql)

rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

In [ ]:
output_path = os.path.join(output_folder, output_csv)
with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([x[0] for x in crsr.description])  # column headers
    for row in rows:
        writer.writerow(row)

print ('csv successfully created at {}'.format(os.path.abspath(output_csv)))